# 🎬 YouTube Video Bot для Google Colab

Этот notebook позволяет запустить Telegram бота для нарезки YouTube видео на шотсы прямо в Google Colab.

## Возможности:
- 📹 Скачивание видео с YouTube в максимальном качестве (раздельно видео + аудио)
- 🍪 Поддержка cookies для приватных и возрастных видео
- ✂️ Автоматическая нарезка на шотсы любой длительности
- 🎤 Генерация субтитров с помощью Whisper AI
- 📱 Красивые заголовки и анимация текста
- ☁️ Автоматическая загрузка на Google Drive
- 🤖 Telegram бот интерфейс
- ⏰ Обработка длинных видео (до нескольких часов)

## Требования:
1. Токен Telegram бота
2. Google OAuth токен для Drive API
3. Cookies файл YouTube (опционально)
4. Стабильное интернет соединение

## 🔧 Шаг 1: Установка зависимостей

In [ ]:
# Обновляем pip и устанавливаем системные зависимости
!apt update -qq
!apt install -y ffmpeg fonts-liberation

# Устанавливаем Python пакеты
!pip install -q python-telegram-bot==20.7
!pip install -q yt-dlp
!pip install -q openai-whisper
!pip install -q google-api-python-client
!pip install -q google-auth-httplib2
!pip install -q google-auth-oauthlib
!pip install -q python-dotenv
!pip install -q Pillow
!pip install -q moviepy

print("✅ Все зависимости установлены!")

## 📥 Шаг 2: Клонирование репозитория

In [ ]:
import os
import shutil

# Удаляем старую папку если есть
if os.path.exists('/content/youtube-video-bot'):
    shutil.rmtree('/content/youtube-video-bot')

# Клонируем репозиторий
!git clone https://github.com/uiper123/BOTTTTTTTNAREZKAAA.git youtube-video-bot

# Переходим в папку проекта
os.chdir('/content/youtube-video-bot')

# Проверяем структуру проекта
!ls -la

print("✅ Репозиторий клонирован!")

## ⚙️ Шаг 3: Настройка токенов

### Получение токенов:

1. **Telegram Bot Token:**
   - Напишите @BotFather в Telegram
   - Создайте нового бота командой `/newbot`
   - Скопируйте токен

2. **Google Drive API:**
   - Перейдите в [Google Cloud Console](https://console.cloud.google.com/)
   - Создайте проект и включите Drive API
   - Создайте OAuth 2.0 credentials
   - Скачайте JSON файл

In [ ]:
# Вставьте ваши токены здесь
TELEGRAM_BOT_TOKEN = "YOUR_TELEGRAM_BOT_TOKEN_HERE"  # Замените на ваш токен
GOOGLE_OAUTH_JSON = """{
  "installed": {
    "client_id": "YOUR_CLIENT_ID",
    "project_id": "YOUR_PROJECT_ID",
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_secret": "YOUR_CLIENT_SECRET",
    "redirect_uris": ["http://localhost"]
  }
}"""  # Замените на ваш Google OAuth JSON

# Проверяем токены
if TELEGRAM_BOT_TOKEN == "YOUR_TELEGRAM_BOT_TOKEN_HERE":
    print("❌ Пожалуйста, установите TELEGRAM_BOT_TOKEN")
else:
    print("✅ Telegram токен установлен")

if "YOUR_CLIENT_ID" in GOOGLE_OAUTH_JSON:
    print("❌ Пожалуйста, установите Google OAuth данные")
else:
    print("✅ Google OAuth данные установлены")

## 🔐 Шаг 4: Настройка Google OAuth

In [ ]:
import base64
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import Flow

# Создаем файл credentials.json
with open('credentials.json', 'w') as f:
    f.write(GOOGLE_OAUTH_JSON)

# Настраиваем OAuth flow
SCOPES = ['https://www.googleapis.com/auth/drive.file']

flow = Flow.from_client_secrets_file(
    'credentials.json',
    scopes=SCOPES,
    redirect_uri='http://localhost'
)

# Получаем URL для авторизации
auth_url, _ = flow.authorization_url(prompt='consent')

print("🔗 Перейдите по этой ссылке для авторизации:")
print(auth_url)
print("\n📋 После авторизации скопируйте код из URL и вставьте ниже:")

# Ждем ввода кода авторизации
auth_code = input("Введите код авторизации: ")

# Получаем токен
flow.fetch_token(code=auth_code)
credentials = flow.credentials

# Сохраняем токен в base64
token_data = pickle.dumps(credentials)
token_base64 = base64.b64encode(token_data).decode('utf-8')

print("✅ Google OAuth настроен успешно!")
print(f"🔑 Токен сохранен (длина: {len(token_base64)} символов)")

## 🍪 Шаг 5: Настройка cookies для YouTube (опционально)

Cookies помогают:
- 🔞 Скачивать возрастные видео
- 🌍 Обходить региональные ограничения
- 🔒 Получать доступ к приватным видео
- 🚫 Избегать блокировок YouTube
- ⚡ Повышать стабильность скачивания

### Как получить cookies:
1. Откройте YouTube в браузере и войдите в аккаунт
2. Установите расширение **"Get cookies.txt LOCALLY"** или **"cookies.txt"**
3. Перейдите на youtube.com
4. Нажмите на иконку расширения
5. Экспортируйте cookies в формате **Netscape**
6. Скопируйте содержимое файла

In [ ]:
# Настройка cookies для YouTube
import os

print("🍪 Настройка cookies для YouTube")
print("Если у вас нет cookies, просто нажмите Enter для пропуска")
print("-" * 50)

use_cookies = input("Хотите использовать cookies? (y/n): ").lower().strip()

if use_cookies in ['y', 'yes', 'да', 'д']:
    print("\n📋 Вставьте содержимое cookies.txt файла:")
    print("(Для завершения ввода введите 'END' на отдельной строке)")
    print("Пример формата:")
    print("# Netscape HTTP Cookie File")
    print(".youtube.com\tTRUE\t/\tFALSE\t1234567890\tname\tvalue")
    print("-" * 30)
    
    cookies_lines = []
    while True:
        line = input()
        if line.strip().upper() == "END":
            break
        cookies_lines.append(line)
    
    if cookies_lines:
        # Сохраняем cookies в файл
        with open('cookies.txt', 'w', encoding='utf-8') as f:
            f.write('\n'.join(cookies_lines))
        
        print(f"✅ Cookies сохранены! ({len(cookies_lines)} строк)")
        
        # Проверяем формат cookies
        valid_cookies = 0
        youtube_cookies = 0
        
        for line in cookies_lines:
            if line.startswith('#') or line.strip() == '':
                continue
            parts = line.split('\t')
            if len(parts) >= 6:
                valid_cookies += 1
                if 'youtube.com' in line or 'google.com' in line:
                    youtube_cookies += 1
        
        print(f"📊 Валидных cookies: {valid_cookies}")
        print(f"🎥 YouTube cookies: {youtube_cookies}")
        
        if youtube_cookies > 0:
            print("✅ YouTube cookies найдены - можно скачивать приватные видео!")
        else:
            print("⚠️ YouTube cookies не найдены - только публичные видео")
    else:
        print("❌ Cookies не введены")
        # Создаем пустой файл
        with open('cookies.txt', 'w') as f:
            f.write('# No cookies\n')
else:
    print("⏭️ Пропускаем настройку cookies")
    # Создаем пустой файл cookies для совместимости
    with open('cookies.txt', 'w') as f:
        f.write('# No cookies\n')

print("\n📁 Проверяем файл cookies:")
if os.path.exists('cookies.txt'):
    file_size = os.path.getsize('cookies.txt')
    print(f"✅ cookies.txt создан ({file_size} bytes)")
    
    # Показываем первые несколько строк (без секретных данных)
    with open('cookies.txt', 'r') as f:
        lines = f.readlines()[:3]
        for i, line in enumerate(lines, 1):
            if line.startswith('#'):
                print(f"  {i}: {line.strip()}")
            else:
                print(f"  {i}: [cookie data]")
else:
    print("❌ cookies.txt не найден")

## 📝 Шаг 6: Создание .env файла

In [ ]:
# Создаем .env файл с настройками
env_content = f"""# Telegram Bot Configuration
TELEGRAM_BOT_TOKEN={TELEGRAM_BOT_TOKEN}

# Google Drive Configuration
GOOGLE_OAUTH_TOKEN_BASE64={token_base64}

# Video Processing Settings
DEFAULT_CLIP_DURATION=30
DEFAULT_TITLE=ФРАГМЕНТ
DEFAULT_SUBTITLE=Часть
WHISPER_MODEL=base
MAX_CHUNK_DURATION=300

# YouTube Cookies (автоматически определяется)
USE_COOKIES={'true' if os.path.exists('cookies.txt') and os.path.getsize('cookies.txt') > 20 else 'false'}
"""

with open('.env', 'w', encoding='utf-8') as f:
    f.write(env_content)

print("✅ Файл .env создан успешно!")
print("📁 Содержимое:")
!head -10 .env

## 🔤 Шаг 7: Загрузка шрифта

In [ ]:
# Скачиваем шрифт для заголовков
import urllib.request

try:
    # Пробуем скачать Roboto Bold (хорошая альтернатива)
    urllib.request.urlretrieve(
        'https://github.com/google/fonts/raw/main/apache/roboto/static/Roboto-Bold.ttf',
        'Obelix Pro.ttf'
    )
    print("✅ Шрифт Roboto Bold загружен успешно!")
except:
    try:
        # Fallback на системный шрифт
        !cp /usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf "Obelix Pro.ttf"
        print("✅ Использован системный шрифт Liberation Sans Bold!")
    except:
        # Последний fallback
        !cp /usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf "Obelix Pro.ttf"
        print("✅ Использован системный шрифт DejaVu Sans Bold!")

# Проверяем наличие шрифта
if os.path.exists('Obelix Pro.ttf'):
    file_size = os.path.getsize('Obelix Pro.ttf')
    print(f"📁 Шрифт готов: Obelix Pro.ttf ({file_size} bytes)")
else:
    print("❌ Ошибка загрузки шрифта")

## 🧪 Шаг 8: Тестирование компонентов

In [ ]:
# Тестируем импорты и основные компоненты
try:
    from youtube_downloader import YouTubeDownloader
    from video_editor import VideoEditor
    from subtitle_generator import SubtitleGenerator
    from google_drive_uploader import GoogleDriveUploader
    from video_processor import VideoProcessor
    
    print("✅ Все модули импортированы успешно!")
    
    # Тестируем создание объектов
    downloader = YouTubeDownloader()
    editor = VideoEditor()
    subtitle_gen = SubtitleGenerator()
    uploader = GoogleDriveUploader()
    processor = VideoProcessor()
    
    print("✅ Все компоненты инициализированы!")
    
    # Проверяем FFmpeg
    print("🎬 Версия FFmpeg:")
    !ffmpeg -version | head -1
    
    # Проверяем Whisper
    import whisper
    print(f"🎤 Whisper доступен, модели: {whisper.available_models()}")
    
    # Проверяем cookies
    if os.path.exists('cookies.txt'):
        with open('cookies.txt', 'r') as f:
            content = f.read()
            if 'youtube.com' in content or 'google.com' in content:
                print("🍪 YouTube cookies активны")
            else:
                print("🍪 Cookies файл создан (без YouTube cookies)")
    
    print("\n🎉 Все компоненты готовы к работе!")
    
except Exception as e:
    print(f"❌ Ошибка тестирования: {e}")
    import traceback
    traceback.print_exc()

## 🚀 Шаг 9: Запуск бота

**Внимание:** После запуска бот будет работать до тех пор, пока вы не остановите ячейку.

### Команды бота:
- `/start` - Начать работу
- `/duration 60` - Установить длительность шотсов (секунды)
- `/title ЭПИЗОД` - Установить заголовок
- `/subtitle Серия` - Установить подзаголовок
- `/settings` - Показать текущие настройки

### Для остановки:
- Нажмите кнопку **"Stop"** или используйте **Ctrl+C**

In [ ]:
# Запускаем бота
print("🤖 Запуск Telegram бота...")
print("📱 Найдите вашего бота в Telegram и отправьте /start")
print("🔗 Отправьте ссылку на YouTube видео для обработки")
print("⏹️ Для остановки нажмите кнопку 'Stop' или Ctrl+C")
print("🍪 Cookies активны - можно скачивать приватные видео" if os.path.exists('cookies.txt') and os.path.getsize('cookies.txt') > 20 else "🍪 Cookies не настроены - только публичные видео")
print("-" * 50)

# Импортируем и запускаем бота
from bot import TelegramBot

try:
    bot = TelegramBot()
    bot.run()
except KeyboardInterrupt:
    print("\n🛑 Бот остановлен пользователем")
except Exception as e:
    print(f"❌ Ошибка запуска бота: {e}")
    import traceback
    traceback.print_exc()

## 📊 Шаг 10: Мониторинг ресурсов (опционально)

In [ ]:
# Мониторинг использования ресурсов
import psutil
import time

def show_resources():
    # CPU
    cpu_percent = psutil.cpu_percent(interval=1)
    
    # Memory
    memory = psutil.virtual_memory()
    memory_percent = memory.percent
    memory_used = memory.used / (1024**3)  # GB
    memory_total = memory.total / (1024**3)  # GB
    
    # Disk
    disk = psutil.disk_usage('/')
    disk_percent = disk.percent
    disk_used = disk.used / (1024**3)  # GB
    disk_total = disk.total / (1024**3)  # GB
    
    print(f"🖥️  CPU: {cpu_percent:.1f}%")
    print(f"🧠 RAM: {memory_percent:.1f}% ({memory_used:.1f}/{memory_total:.1f} GB)")
    print(f"💾 Disk: {disk_percent:.1f}% ({disk_used:.1f}/{disk_total:.1f} GB)")
    
    # Проверяем файлы в temp и output
    if os.path.exists('temp'):
        temp_files = len([f for f in os.listdir('temp') if os.path.isfile(os.path.join('temp', f))])
        temp_size = sum(os.path.getsize(os.path.join('temp', f)) for f in os.listdir('temp') if os.path.isfile(os.path.join('temp', f))) / (1024**2)
        print(f"📁 Temp files: {temp_files} ({temp_size:.1f} MB)")
    
    if os.path.exists('output'):
        output_files = len([f for f in os.listdir('output') if os.path.isfile(os.path.join('output', f))])
        print(f"📤 Output files: {output_files}")

# Показываем текущее состояние
show_resources()

## 🧹 Шаг 11: Очистка временных файлов

In [ ]:
# Очистка временных файлов для освобождения места
import shutil

def cleanup_temp_files():
    cleaned_size = 0
    
    # Очищаем temp папку
    if os.path.exists('temp'):
        for file in os.listdir('temp'):
            file_path = os.path.join('temp', file)
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path)
                os.remove(file_path)
                cleaned_size += size
        print(f"🗑️ Очищена папка temp")
    
    # Очищаем кэш Whisper
    whisper_cache = os.path.expanduser('~/.cache/whisper')
    if os.path.exists(whisper_cache):
        cache_size = sum(os.path.getsize(os.path.join(whisper_cache, f)) 
                        for f in os.listdir(whisper_cache) if os.path.isfile(os.path.join(whisper_cache, f)))
        shutil.rmtree(whisper_cache)
        cleaned_size += cache_size
        print(f"🗑️ Очищен кэш Whisper")
    
    # Очищаем кэш yt-dlp
    ytdlp_cache = os.path.expanduser('~/.cache/yt-dlp')
    if os.path.exists(ytdlp_cache):
        shutil.rmtree(ytdlp_cache)
        print(f"🗑️ Очищен кэш yt-dlp")
    
    print(f"✅ Освобождено: {cleaned_size / (1024**2):.1f} MB")

# Запускаем очистку
cleanup_temp_files()

# Показываем обновленное состояние ресурсов
print("\n📊 Состояние после очистки:")
show_resources()

## 💡 Полезные советы

### 🍪 Использование cookies:
- **Возрастные видео**: Cookies позволяют скачивать 18+ контент
- **Приватные видео**: Доступ к unlisted и private видео
- **Региональные ограничения**: Обход geo-блокировок
- **Стабильность**: Меньше ошибок при скачивании

### ⚡ Для оптимальной работы:
1. **Используйте GPU runtime** для ускорения Whisper
2. **Периодически очищайте temp файлы** для экономии места
3. **Мониторьте использование RAM** при обработке длинных видео
4. **Обновляйте cookies** если они перестали работать

### 🎬 Обработка видео:
- **Короткие видео** (до 5 мин): Обрабатываются целиком
- **Длинные видео** (5+ мин): Автоматически разбиваются на чанки
- **Часовые видео**: Могут занять 1-2 часа обработки
- **Качество**: Раздельное скачивание видео+аудио = максимальное качество

### 🚨 Ограничения Google Colab:
- **Время сессии**: ~12 часов максимум
- **Место на диске**: ~100 GB
- **RAM**: 12-25 GB (зависит от runtime)
- **Сеть**: Может быть ограничена при интенсивном использовании

### 🔧 Решение проблем:
- **"Out of memory"**: Уменьшите длительность чанков или используйте меньшую Whisper модель
- **"Disk space full"**: Регулярно очищайте временные файлы
- **"Session timeout"**: Сохраните прогресс и перезапустите notebook
- **"Cookies expired"**: Обновите cookies файл
- **"YouTube blocking"**: Используйте свежие cookies или VPN

### 📱 Команды бота:
```
/start - Начать работу
/duration 60 - Установить длительность шотсов (секунды)
/title ЭПИЗОД - Установить заголовок
/subtitle Серия - Установить подзаголовок
/settings - Показать текущие настройки
/help - Помощь
```

### 🎯 Примеры использования:
- **Подкасты**: `/duration 120` для 2-минутных отрывков
- **Лекции**: `/title УРОК` `/subtitle Часть` для образовательного контента
- **Интервью**: `/duration 45` для коротких цитат
- **Музыка**: Используйте cookies для доступа к возрастному контенту